### <font color='darkblue'>Week 8 Computing Lab</font>

#### Working with Onotlogies and Functional Enrichment Analysis

##### (and a little bit of network analysis..)

##### <font color='darkblue'>Introduction</font>
In this computing lab we’re going to be putting together what we've learned about biological databases and ontologies to do some summary analysis of genes invovled in the "Dopaminergnic Synapse" pathway. These can all be done using the KEGG and String-DB websites directly but we will show here that there is much greater power and flexibility available when you start using programmatic methods to carefully control your analyses.

##### <font color='darkblue'>Learning Outcomes</font>
After this tutorial you should be comfortable with:
- Retrieving pathway information from KEGG
- converting between accession IDs of different databases
- Retrieving protein-protein interaction and network data from String-DB
- Automating these processes using APIs and the NetworkX, and GSEAPy python packages

## Step 1 - Setting up the Environment & Retrieving Data 

In [ ]:
# Setting Up the Programming Environment
# %pip install networkx
# %pip install gseapy

# import modules for use in the notebook

# handling www based requests (like APIs)
import urllib as ul

# standard Python data handling modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# working with networks
import networkx as nx

# working with geen set enrichment analysis (GSEA)
import gseapy

For Step1 you can either use the fully automated approcah using Steps 1a, and 1b, or working from files you generate from the KEGG and String-DB websites as described in Step 1c.

### Step 1a - Automating Download of KEGG Pathway Information

In [ ]:
# First we fetch the list of human pathways available at KEGG.

human_pathways = pd.read_csv(ul.request.urlopen('http://rest.kegg.jp/list/pathway/hsa'),sep='\t',header=0,names=['kegg_id','pathway_name'])
human_pathways.head()

# We specifically want the pathway data for the "Dopaminergic Synapse" pathway.
pathway_info = human_pathways[human_pathways['pathway_name'].str.match('Dopaminergic synapse')]['kegg_id']

# extract the exact pathway accession
pathway_id = pathway_info.values[0].split(':')[1]

print(pathway_id)

# pull the pathway directly from KEGG, note we are saving this to a file 'dop_synapse.txt' that we will use later
ul.request.urlretrieve('http://rest.kegg.jp/get/'+pathway_id,'dop_synapse.txt');

In [ ]:
# Now we will use this file which contains the full pathway details including the gene names.

# open the file
dop_file = open('dop_synapse.txt','r')

# I wanted to show you some basic python parsing and a simple for loop with a conditional in to demonstrate how you can quickly build simple parsers.
# There are quicker ways to do this, but this is a good learning example.

# create an empty dataframe
dop_df = pd.DataFrame()

# set a flag for our parser
flag=0

# work through the text file one line at a time
for line in dop_file:
    # find the start of the gene entries
    if 'GENE' in line:
        # add the first gene tp the dataframe
        dop_df = dop_df.append(pd.Series(line.strip('GENE').strip().split('  ')),ignore_index=True)
        # set the flag to 1, we are in the gene section of the file
        flag = 1
    # stop when we reach the end of the section and escape the file
    elif 'COMPOUND' in line:
        break
    # continue adding the genes to the dataframe
    elif flag == 1:
        dop_df = dop_df.append(pd.Series(line.strip().split('  ',2)),ignore_index=True)

# close the file
dop_file.close()

# name the columns
dop_df.columns = ['gene_id','description']

# view the file
dop_df.head()

# you now have the gene_ids (NCBI EntrezIDs for the genes in the pathway)
print('The Dopaminergic Synapse pathway has '+str(dop_df.shape[0])+' genes in it.\n')

# show the gene_ids
print(dop_df['gene_id'].to_numpy())

### Step 1b - Automating Retrieval of Protein-Protein Interactions from STRING

The details of the String-DB API can be found here - [https://string-db.org/help/api/](https://string-db.org/help/api/)

APIs have specific formats required for their query URLs and it getting these correct in your code can take a little time until you get used to them. In this case we need to concatenate (stitch together) our gene IDs using a '%0D' string. This is actually the encoding for a line-return which is in effect mimicking the one gene per line entry that you would paste into the web page.

In [ ]:
# create a concatenated list of entrezIDs as strings
# note we are taking integer gene_ids from the 'gene_id' column of the dataframe we generated above then using
# the map function to convert each one into a string. The join function then concatenates them using the '%0D' string
# to stitch them all together. This string will be used to help us build the API query URL.
entrezIDs = '%0D'.join(map(str,dop_df['gene_id']))

# pass the list of EntrezIDs to the String-DB API return the String-IDs
# we first form the query url using the 'get_string_ids' API function which takes a list of identifiers and
# converts them into the internal String-DB accession IDs. This massively speeds up the search and allows us to
# search for more than 10 at once which is an API restriction for other API functions if String-DB internal accessions 
# aren't used.
query_url = 'https://string-db.org/api/tsv-no-header/get_string_ids?identifiers='+entrezIDs+'&format=only-ids'

# use the urllib library to retrieve the String-DB internal IDs
result = ul.request.urlopen(query_url).read().decode('utf-8')

# now we want to query String-DB to retrieve interactions from this list of String-DB IDs
# we create a concatenated list of stringdbIDs in much the same way as above for the Entrez Gene IDs
stringdbIDs = '%0D'.join(result.splitlines())

# again we build the query for interactions using the String-DB IDs
query_url = 'https://string-db.org/api/tsv/network?identifiers='+stringdbIDs+'&species=9606'

# again using urllib to retrieve the interactions these are returned in a standard tab delimied text format
interactions = ul.request.urlopen(query_url).read().decode('utf-8').splitlines()

# we need to split the result by these 'tabs' (\t - is used to identfy them)
int_test = [interaction.split('\t') for interaction in interactions]

# we extract the field names from the first row
column_names = int_test[:1][0]

# create a Pandas dataframe of the interaction data we have just retrieved from String-DB
interactions_df = pd.DataFrame(int_test,columns=column_names)

# delete the first row that held the fieldnames but we no longer need
interactions_df = interactions_df.drop(labels=0,axis=0)

# remove any duplicate rows
final_interactions = interactions_df.drop_duplicates()

# show the top of the protein-protein interaction table
final_interactions.head()

### Step 1c - Download Network from [STRING](https://string-db.org/cgi/input?sessionId=bIvRxxC0rWvS&input_page_show_search=on)

In string upload dop_geneids.txt to string (make sure to click on multiple proteins)

Select Homo Sapiens in Organism and click Search

Quickly double check the mapping is correct before clicking continue

You should now see your full protein-protein interaction network

What we would like to do is to identify clusters of enriched terms in this network

First step is to cluster the network. This can be achieved in STRING by selecting Clusters -> MCL Clustering

Change edge between clsuters to solid line

The next step is to download our clustered network. 

This is achieved by selceting Exports -> '... as tabular text output' 

This file is the full network with edges between every node. It is a '.tsv' format which can work with in Python

## Step 2 - Generating the Protein-Protein Interaction Network

Next we are going to use the NetworkX Python library to create the protein-protein interaction network.

NetworkX - Network Analysis in Python - [https://networkx.org/documentation/stable/index.html](https://networkx.org/documentation/stable/index.html)

### Step 2a - Working with the PPI dataframe created in Step 1b

In [ ]:
# check the column names of the dataframe
print(final_interactions.columns)

 #Create an empty graph
G = nx.Graph()

# add all nodes
G.add_nodes_from(set(final_interactions['preferredName_A']) | set(final_interactions['preferredName_B'])) 

# add the edges (connections) to the network
edges = []
for edge1 , edge2  in zip(final_interactions['preferredName_A'] , final_interactions['preferredName_B']) : #add all edge to the network
    edges.append((edge1 , edge2 ))
G.add_edges_from(edges)

# draw the network
nx.draw(G , with_labels = True)

### Step 2b - Working from the PPI file downloaded from the StringDB website in Step 1c

In [ ]:
# read in the protein-protein interaction data from the downloaded String-DB tab separated file
df = pd.read_csv('string_interactions.tsv' , delimiter='\t')

# show the top of the table
df.head()

In [ ]:
# check the column names of the dataframe
# print(df.columns)

 #Create an empty graph
G = nx.Graph()

# add all nodes. Note we need to cover the scenario where node2 is connected to node1 only
G.add_nodes_from(set(df['#node1']) |  set(df['node2']))

# add the edges (connections) to the network
edges = []
for edge1 , edge2  in zip(df['#node1'] , df['node2']) :
    edges.append((edge1 , edge2 ))

G.add_edges_from(edges)

# draw the network
nx.draw(G , with_labels = True)

#note how we keep our gene ids in the node names
print(G.nodes)

## Step 3 - Gene Set Enrichement Analysis for our Network

### [GSEApy](https://gseapy.readthedocs.io/en/latest/index.html)

[GSEApy](https://gseapy.readthedocs.io/en/latest/index.html) is a library to perform gene set enrichment analysis (GSEA) in python. There are two methods to perform enrichment analysis - over representation analysis and GSEA. The main difference between the two is that GSEA assumes your input list of genes is ordered by the most representative genes in that list. 

### Enrichment Analysis Measures

In order to perform GSEA we need to impose an ordering on the list of gene ids. We want to order genes such that the genes which are most important to the above network are listed highest. 

Important genes to the network are ones that are central to the network structural i.e. removing these nodes will split the network or cause it to lose its structure. This means they tend to have highest number of connections and located in the centres of clusters.

You can browse the available gene sets to perform enrichment analysis against using the GSEAPy package at the [Enrichr website](https://maayanlab.cloud/Enrichr/#libraries).

### Step 3a - Node Degree

Node degree is simply the number of connections a node has. The higher the number of connections, the more important the node

In [ ]:
#sort the genes (node names) by degree
sorted_list = sorted(G.degree(), key=lambda item: item[1] , reverse=True)

#extract just the node names from the sorted list
sorted_genes = []
for item in sorted_list :
    sorted_genes.append(item[0])
    
# perform enrichment analysis using gsea
enr = gseapy.enrichr(gene_list=sorted_genes,
                 gene_sets=['KEGG_2021_Human'],
                 organism='human', # don't forget to set organism to the one you desired! e.g. Yeast
                 outdir=None, # don't write to disk
                )

enr.results.head(10) #return the top 10 hits

### Step 3b - Closeness Centrality 
This is a measure of how close a node is to the center of the network. The closer a node is to the center the shorter its path to all other nodes and hence its more likely to be representative of the network

In [ ]:
sorted_list = sorted(nx.closeness_centrality(G).items(), key=lambda item: item[1] , reverse=True) 
#sort the genes (node names) by proximity to center

sorted_genes = []
for item in sorted_list : #extract just the node names from the sorted list
    sorted_genes.append(item[0])
    
enr = gseapy.enrichr(gene_list=sorted_genes, # perform enrichment analysis using gsea
                 gene_sets=['KEGG_2021_Human'],
                 organism='human', # don't forget to set organism to the one you desired! e.g. Yeast
                 outdir=None, # don't write to disk
                )

enr.results.head(10) #return the top 10 hits

### Step 3c - Clustering Coefficient
The clustering coefficient is a measure which combines centrality and degree. It measures the number of triangles a node can form ('the friend of my friend is my friend'). If a node has more common friends with other nodes it more likely to representative of the network

In [ ]:
sorted_list = sorted(nx.clustering(G).items(), key=lambda item: item[1] , reverse=True)
#sort the genes (node names) by clustering coefficient

sorted_genes = []
for item in sorted_list : #extract just the node names from the sorted list
    sorted_genes.append(item[0])
    
enr = gseapy.enrichr(gene_list=sorted_genes, # perform enrichment analysis using gsea
                 gene_sets=['KEGG_2021_Human'],
                 organism='human', # don't forget to set organism to the one you desired! e.g. Yeast
                 outdir=None, # don't write to disk
                )

enr.results.head(10) #return the top 10 hits

## Step 4 - Exploring Gene Ontology Annotations

In this step we use biomart which is an excellent service run at EBI-Ensembl that allows you to query and retrieve linked data for genomic data.

The help for Biomart can be found here - [https://www.ensembl.org/info/data/biomart/how_to_use_biomart.html](https://www.ensembl.org/info/data/biomart/how_to_use_biomart.html)

Biomart API functionality is nicely delivered through GSEApy and its use is described here - [https://gseapy.readthedocs.io/en/latest/gseapy_example.html#1.-Biomart-API](https://gseapy.readthedocs.io/en/latest/gseapy_example.html#1.-Biomart-API).

In [ ]:
# read in the Entrez Gene IDs for the Dopaminergic Synapse pathway
# **you could pull the pathway details as in Step 1a above.
gene_ids = pd.read_table('dop_geneids.txt' , header=None , names=['gene_ids'])

gene_ids.head()

In [ ]:
# the GSEApy package also contains functions that allow you to use the EBI-Ensembl Biomart service
# you can use this to directly query linked data for the genes, including Gene Ontology (GO) annotation data.
from gseapy import Biomart

# initiate a biomart connection
bm = Biomart()

# form a query for biomart from the Entrez Gene IDs
queries = {'entrezgene_id' :list(gene_ids.values.reshape(1,-1)[0])}

# execute the biomart query
# NB that the oddly named 'name_1006' attribute is in fact the 'GO Term name' attribute
# NB a nice trick for finding the correct attribute names is to use the website to create the query and then
# NB click on the XML link at the top of the page, this will show you the 'Attribute name'.
results = bm.query(dataset='hsapiens_gene_ensembl',
                   attributes=['ensembl_gene_id', 'external_gene_name', 'entrezgene_id', 'go_id','go_linkage_type','name_1006'],
                   filters=queries)

# change the name to something more useful
results.rename({'name_1006' : 'go_name'},axis=1,inplace=True)

results.head()

In [ ]:
#Plot the top 20 most common GO terms
top_20_GO = results['go_name'].value_counts()[:20];

top_20_GO.plot.bar(xlabel='GO Term',ylabel='GO Term Frequency',title='Top 20 Most Frequent GO Annotations in the Dopaminergic Synapse Pathway');


In [ ]:
# just to take extra advantage of the data lets break down the above by evidence code type
annotations_by_evidence = pd.DataFrame(results[['go_name','go_linkage_type']].dropna())

# we use the Pandas pivot_table function to count and sort the pairs go_term & evidnce code
annotations_by_evidence = annotations_by_evidence.pivot_table(index=['go_name','go_linkage_type'],aggfunc='size').reset_index(name='frequency').sort_values(by='frequency',ascending=False)

# just take the top 30 rows
top_annotations = annotations_by_evidence[:30]

# pivot to create a nice matrix for a stacked plot
top_annotations = top_annotations.pivot(index='go_name',columns='go_linkage_type',values='frequency').fillna(0)

# create a stacked barchart of the results
top_annotations.plot.bar(xlabel='GO Term',
    ylabel='GO Term Frequency',
    title='Top Frequent GO Annotations by Evidence Code in the Dopaminergic Synapse Pathway',
    stacked=True).legend(loc='best');

# the nicely reshaped data frame used for the stacked plot
top_annotations.head()

Check the function of the most common GO terms [here](https://www.ebi.ac.uk/QuickGO/)

## Step5 - [PantherDB](http://www.pantherdb.org/)

[PantherDB](http://www.pantherdb.org/) is another online tool which can be used to perform an enrichment analysis.

### Step 5a - Using the PantherDB Website for Enrichment Analysis

In [ ]:
# generate a unique list of Dopaminergic Synapse genes from our earlier dataframe
dop_genes = list(dop_df['gene_id'].dropna().drop_duplicates())

# write them to a file to upload to the PantherDB website
with open('entrezgene_id.txt', 'w') as f:
    f.write('\n'.join(str(gene) for gene in dop_genes))

Upload this file on the [PantherDB](http://www.pantherdb.org/) website

Select Functional classification viewed in graphic charts bar chart and be sure to select "Homo sapiens" as species

Do the GO terms match up to that found in PantherDB, why or why not? 

### Step 5b - Using the PantherDB API for Enrichment Analysis

PantherDB API details - [http://pantherdb.org/services/details.jsp](http://pantherdb.org/services/details.jsp)

Functionality and Parameter testing - [http://pantherdb.org/services/openAPISpec.jsp](http://pantherdb.org/services/openAPISpec.jsp)

This is quite hard work so you might decide it's simplest (and quicker) at this stage to use the website functionality above).

In [ ]:
# results are returned in JSON format so we need to load a Python module to handle this
import json

# the PantherDB API offers this function to find out what annotated resources it has available
query_url = 'http://pantherdb.org/services/oai/pantherdb/supportedannotdatasets'

# execute the query
result = ul.request.urlopen(query_url)

# load the results returning a Python dictionary
annotationSets = json.load(result)

annotations = annotationSets['search']['annotation_data_sets']['annotation_data_type']

# we can just iterate through these to see the annotation sources available
for i in annotations:
    print('Annotation Set Label = '+i['label']+', annotDataSet string to use below = '+i['id'])

In [ ]:
# using the list of Entrez Gene IDs generated above (entrezgene_id) create a query string for the API
# these need to be comma separated
genes = ','.join(map(str,dop_genes))

# use the PantherDB API - NB that GO:0008150 is the accession for the "Biological Process" clade of the Gene Ontology from above
query_url = "http://pantherdb.org/services/oai/pantherdb/enrich/overrep?&geneInputList="+genes+"&organism=9606&annotDataSet=GO:0008150&enrichmentTestType=FISHER&correction=FDR"

# capture the results (NB this returns in JSON format)
result = ul.request.urlopen(query_url)

# load the results from JSON to Python dictionary
enrichment_result = json.load(result)

# view the raw results
print(enrichment_result)

We're now going to format that into something human readable. There are many ways to do this, but this is a quick and (fairly) simple solution. Please do feel free to try your own.

In [ ]:
# extract the actual result component
results = enrichment_result['results']['result']

# how long is the background list (in this case it is the default, the whole genome)
print(len(results), "terms in reference list")

# print some headers
display_headers = ["GO Term", "Expected", "Fold enrichment", "raw P value", "FDR", "Term label"]
print("\t".join(display_headers))

# Sort in order of false discovery rate i.e. multiple testing correction
results.sort(key=lambda x: x['fdr'], reverse=False)

# iterate through all of the results and print out the biots we're interested in
for r in results:
    fdr = r['fdr']
    if fdr < 0.05:
        # Print result line
        term_id = r['term'].get("id")
        if term_id is None:
            term_id = ""
        else:
            print("\t".join([
                term_id,
                str(r['expected']),  # Convert float to string for printing
                str(r['fold_enrichment']),
                str(r['pValue']),
                str(r['fdr']),
                r['term']["label"]
            ])
            )

# that was quite painful